In [2]:
# https://drive.google.com/file/d/1lKeP9E_S8uuaMu12lv8snzVKXXHG99ck/view?usp=sharing
%matplotlib inline
import torch
print(torch.__version__)
!pip install torch-cluster torch-geometric -f https://data.pyg.org/whl/torch-{torch.__version__}.html

2.1.0+cu118
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html


In [2]:
!gdown 1lKeP9E_S8uuaMu12lv8snzVKXXHG99ck

Downloading...
From: https://drive.google.com/uc?id=1lKeP9E_S8uuaMu12lv8snzVKXXHG99ck
To: /content/IMDB-Movie-Data.csv
100% 310k/310k [00:00<00:00, 3.92MB/s]


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("./IMDB-Movie-Data.csv")

df.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


In [ ]:
#  Title, Genre, Director, Actors and Years
K2I = {
    "Title": 0,
    "Genre": 1,
    "Director": 2,
    "Actors": 3,
    "Year": 4
}
KEYS = [key for key in K2I]
ACTORS = []
GENRES = []
DIRS = []
for i,row in df.iterrows():
  GENRES.extend(row["Genre"].split(","))
  DIRS.append(row["Director"])
  ACTORS.extend(row["Actors"].split(","))

ACTORS = list(dict.fromkeys(ACTORS))
GENRES = list(dict.fromkeys(GENRES))
DIRS = list(dict.fromkeys(DIRS))

print("Actors: ", ACTORS)
print("Genres: ", GENRES)
print("Directors: ", DIRS)

def extract(key: str, row: list):
  if(key == "Title"):
    return row["Rank"]
  if(key == "Director"):
    return DIRS.index(row[key])
  if(key in ["Year"]):
    return row[key]
  if(key in ["Genre", "Actors"]):
    parts = row[key].split(",")
    l = GENRES if key == "Genre" else ACTORS
    return [1 if item in parts else 0 for item in l]

# def connect(n1, n2, key):
#   f1 = n1[K2I[key]]
#   f2 = n2[K2I[key]]

#   if(type(key) is str):
#     if(f1 == f2):
#       return True
#     else: return False

#   if(len(set(f1).intersection(set(f2))) > 0):
#     return True
#   return False


def connect(n1, n2):
  for i in range(len(n1)):
    if(n1[i] == n2[i]):
      return True
  return False

def getEmanating(edges, node):
  ret = []
  for edge in edges:
    if(node == edge[0]):
      ret.append(edge[1])
    elif(node == edge[1]):
      ret.append(edge[0])

  return ret

edges = []
features = []
for i, row in df.iterrows():
  f = []
  for key in KEYS:
    t = extract(key, row)
    if(type(t) is list):
      f.extend(t)
    else:
      f.append(t)

  features.append(f)
  if(i <4): print(f)

for i, n1 in enumerate(features):
  for j, n2 in enumerate(features):
    if(i>=j): continue
    edge = connect(n1, n2)
    if(edge):
      edges.append([i,j])




Actors:  ['Chris Pratt', ' Vin Diesel', ' Bradley Cooper', ' Zoe Saldana', 'Noomi Rapace', ' Logan Marshall-Green', ' Michael Fassbender', ' Charlize Theron', 'James McAvoy', ' Anya Taylor-Joy', ' Haley Lu Richardson', ' Jessica Sula', 'Matthew McConaughey', 'Reese Witherspoon', ' Seth MacFarlane', ' Scarlett Johansson', 'Will Smith', ' Jared Leto', ' Margot Robbie', ' Viola Davis', 'Matt Damon', ' Tian Jing', ' Willem Dafoe', ' Andy Lau', 'Ryan Gosling', ' Emma Stone', ' Rosemarie DeWitt', ' J.K. Simmons', 'Essie Davis', ' Andrea Riseborough', ' Julian Barratt', 'Kenneth Branagh', 'Charlie Hunnam', ' Robert Pattinson', ' Sienna Miller', ' Tom Holland', 'Jennifer Lawrence', ' Chris Pratt', ' Michael Sheen', 'Laurence Fishburne', 'Eddie Redmayne', ' Katherine Waterston', ' Alison Sudol', 'Dan Fogler', 'Taraji P. Henson', ' Octavia Spencer', ' Janelle Monáe', 'Kevin Costner', 'Felicity Jones', ' Diego Luna', ' Alan Tudyk', ' Donnie Yen', "Auli'i Cravalho", ' Dwayne Johnson', ' Rachel Hou

In [ ]:
import os.path as osp
import sys

import matplotlib.pyplot as plt
import torch
from sklearn.manifold import TSNE

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec
from torch_geometric.data import Data



edge_index = torch.tensor(edges, dtype=torch.long)
x = torch.tensor(features, dtype=torch.float)

dataset = Data(x=x, edge_index=edge_index.t().contiguous())
data = dataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Node2Vec(
    data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,
    q=1.0,
    sparse=True,
).to(device)

num_workers = 4
loader = model.loader(batch_size=128, shuffle=True, num_workers=num_workers)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


for epoch in range(1, 101):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 001, Loss: 7.1173
Epoch: 002, Loss: 5.9045
Epoch: 003, Loss: 5.3359
Epoch: 004, Loss: 4.8217
Epoch: 005, Loss: 4.3983
Epoch: 006, Loss: 4.0105
Epoch: 007, Loss: 3.6645
Epoch: 008, Loss: 3.3475
Epoch: 009, Loss: 3.0634
Epoch: 010, Loss: 2.8065
Epoch: 011, Loss: 2.5682
Epoch: 012, Loss: 2.3756
Epoch: 013, Loss: 2.2048
Epoch: 014, Loss: 2.0482
Epoch: 015, Loss: 1.9134
Epoch: 016, Loss: 1.7842
Epoch: 017, Loss: 1.6788
Epoch: 018, Loss: 1.5828
Epoch: 019, Loss: 1.5011
Epoch: 020, Loss: 1.4203
Epoch: 021, Loss: 1.3584
Epoch: 022, Loss: 1.2995
Epoch: 023, Loss: 1.2383
Epoch: 024, Loss: 1.1938
Epoch: 025, Loss: 1.1561
Epoch: 026, Loss: 1.1152
Epoch: 027, Loss: 1.0816
Epoch: 028, Loss: 1.0528
Epoch: 029, Loss: 1.0281
Epoch: 030, Loss: 1.0045
Epoch: 031, Loss: 0.9865
Epoch: 032, Loss: 0.9693
Epoch: 033, Loss: 0.9533
Epoch: 034, Loss: 0.9415
Epoch: 035, Loss: 0.9292
Epoch: 036, Loss: 0.9187
Epoch: 037, Loss: 0.9077
Epoch: 038, Loss: 0.8983
Epoch: 039, Loss: 0.8916
Epoch: 040, Loss: 0.8853


In [ ]:
# import torch
# from sklearn.metrics import roc_auc_score

# import torch_geometric.transforms as T
# from torch_geometric.nn import GCNConv
# from torch_geometric.utils import negative_sampling
# from torch_geometric.data import Data

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# transform = T.Compose([
#     # T.NormalizeFeatures(),
#     T.ToDevice(device),
#     T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
#                       add_negative_train_samples=False),
# ])
# edge_index = torch.tensor(edges, dtype=torch.long)
# x = torch.tensor(features, dtype=torch.float)

# dataset = Data(x=x, edge_index=edge_index.t().contiguous(), transform=transform)
# print(dataset)
# train_data = dataset

# class Net(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super().__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, out_channels)

#     def encode(self, x, edge_index):
#         x = self.conv1(x, edge_index).relu()
#         return self.conv2(x, edge_index)

#     def decode(self, z, edge_label_index):
#         return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

#     def decode_all(self, z):
#         prob_adj = z @ z.t()
#         return (prob_adj > 0).nonzero(as_tuple=False).t()


# model = Net(dataset.num_features, 128, 64).to(device)
# optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
# criterion = torch.nn.BCEWithLogitsLoss()


# def train():
#     model.train()
#     optimizer.zero_grad()
#     z = model.encode(train_data.x, train_data.edge_index)

#     # We perform a new round of negative sampling for every training epoch:
#     neg_edge_index = negative_sampling(
#         edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
#         num_neg_samples=train_data["edge_index"].size(1), method='sparse')

#     # Concat positive and negative edge indices.
#     edge_label_index = torch.cat(
#         [train_data["edge_index"], neg_edge_index],
#         dim=-1,
#     )
#     # Label for positive edges: 1, for negative edges: 0.
#     edge_label = torch.cat([
#         train_data.edge_label,
#         train_data.edge_label.new_zeros(neg_edge_index.size(1))
#     ], dim=0)

#     # Note: The model is trained in a supervised manner using the given
#     # `edge_label_index` and `edge_label` targets.
#     out = model.decode(z, edge_label_index).view(-1)
#     loss = criterion(out, edge_label)
#     loss.backward()
#     optimizer.step()
#     return loss


# # @torch.no_grad()
# # def test(data):
# #     model.eval()
# #     z = model.encode(data.x, data.edge_index)
# #     out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
# #     return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

# # Train/Test Loop
# best_val_auc = final_test_auc = 0
# for epoch in range(1, 101):
#     loss = train()
#     # val_auc = test(val_data)
#     # test_auc = test(test_data)
#     # if val_auc > best_val_auc:
#         # best_val_auc = val_auc
#         # final_test_auc = test_auc
#     print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

# print(f'Final Test: {final_test_auc:.4f}')

# # z = model.encode(test_data.x, test_data.edge_index)
# # final_edge_index = model.decode_all(z)

Data(x=[1000, 2417], edge_index=[2, 499500], transform=Compose([
  ToDevice(cpu),
  RandomLinkSplit(num_val=0.05, num_test=0.1)
]))


AttributeError: ignored